1. Make sure only relevant acorns are in the data
2. Make a boxplot 
3. Use the data without anomalies and calculate the average energy per day
4. Merge this with the anomalies data and make a new column for average energy per day without the anomalies
5. Save anomalies in excel file
6. Do an if-else statement in excel to get over or under consumption
7. split file into 3 part (go as far a here)
-save a csv file with the over and under consumption anomalies
-save a csv file with only over consumption anomalies
-save a csv file with only under consumption anomalies
8. Do visualizations

# Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from sklearn.ensemble import IsolationForest
from pylab import rcParams

In [ ]:
#setting visualisation size
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 14, 8

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)

# Affluent Achievers

In [ ]:
# import combined file
# importing daily dataset files
block_0 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_0.csv")
block_1 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_1.csv")
block_2 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_2.csv")
block_3 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_3.csv")
block_4 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_4.csv")
block_5 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_5.csv")
block_6 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_6.csv")
block_7 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_7.csv")
household_acorn_info = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\informations_households_Cleaned.csv")

In [ ]:
# checking number of rows in first file
len(block_0)

In [ ]:
# adding all files in one data frame
Dfs = [block_0, block_1, block_2, block_3, block_4, block_5, block_6,block_7]
Combined = pd.concat(Dfs)

In [ ]:
# checking number of rows in combined dataframe
len(Combined)

In [ ]:
# checking the combined dataframe
Combined.head(5)

In [ ]:
# renaming columns
Combined.rename(columns = {"day" : "date"}, inplace = True)
household_acorn_info.rename(columns = {"day" : "date"}, inplace = True)

In [ ]:
# merging combined dataframe with household acorn details to see which acorn each household_id is associated with
Combined =  Combined.merge(household_acorn_info,on='Household_Id').reset_index()

In [ ]:
# checking if only acorns that belong to Affluent Achievers are in the dataframe 
Combined.Acorn.unique()

In [ ]:
# Households belonging to acorn D were included so we removed them as they don't belong to Affluent Achievers
Combined=Combined.loc[Combined['Acorn']!='ACORN-D']
Combined.head(5)

In [ ]:
# checking if only acorns A,B and C remain in the dataframe
Combined.Acorn.unique()

In [ ]:
len(Combined)

In [ ]:
# boxplot with outliers for Affluent Achivers
Energy_Sum_BP= sns.boxplot(x= Combined["energy_sum"])
Energy_Sum_BP.set_xlim(0,500)
plt.title("Affluent Achievers Sum Energy Boxplot")
plt.xlabel("Total Kilowatts Per Day")

In [ ]:
# rounding energy_sum to 2 decimal places
Combined.energy_sum = Combined.energy_sum.round(2)
Combined.head(5)

In [ ]:
# skewness and kurtosis values of energy sum column
print ("Skew: %f"%Combined["energy_sum"].skew())
print ("Kurt: %f"%Combined["energy_sum"].kurt())

In [ ]:
# checking how many null values are in the energy_sum column
Combined.isnull().sum()

In [ ]:
# energy sum is our target value for anomaly detection and it only had one empty value so we removed it
# we decided not to replace it with the mean as energy sum is our target variable for anomlay detection
# we did not think it was approprate to replace it with he mean because that household might have actually been an anomaly
# as it was only one value that was empty, we decided to remove it
Combined = Combined[Combined['energy_sum'].notna()]
len(Combined)

In [ ]:
# checking if the null value in the energy sum column was removed
Combined.isnull().sum()

In [ ]:
# running the Isolation Forest algorithm on the energy_sum column
aa_ad=IsolationForest(n_estimators=60, max_samples='auto', contamination=float(0.1),max_features=1.0)
aa_ad.fit(Combined[['energy_sum']])

In [ ]:
# adding score and anomaly columns to the dataframe
# if the anomaly column has a 1 that means the row is not an anomaly 
# if the anomaly column has a -1 that means the row is an anomaly
Combined['Score']=aa_ad.decision_function(Combined[['energy_sum']])
Combined['Anomaly']=aa_ad.predict(Combined[['energy_sum']])
Combined.head(5)

In [ ]:
# storing the anomalies in a new dataframe
anomalies=Combined.loc[Combined['Anomaly']==-1].reset_index()
anomalies =anomalies.drop(['index'],axis=1)
anomalies.head(5)

In [ ]:
# checking how many rows were anomalies
len(anomalies)

In [ ]:
# sorting the new dataframe in descending order
anomalies.sort_values(by=["energy_sum"], ascending=False).reset_index()

In [ ]:
# percentage of data that are anomalies
print("Percentage of data that are anomalies:",round(len(anomalies)/len(Combined)*100,1),"%")

In [ ]:
anomalies.to_csv(r'C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\AA_anomalies.csv', index = False)

In [ ]:
# storing the data that are not anomalies in a new dataframe
anomalies_removed_data=Combined.loc[Combined['Anomaly']!=-1].reset_index()
anomalies_removed_data =anomalies_removed_data.drop(['index'],axis=1)
anomalies_removed_data.head(5)

In [ ]:
# checking how many rows were not anomalies
len(anomalies_removed_data)

In [ ]:
# boxplot without anomalies
Energy_Sum_BP_RD= sns.boxplot(x= anomalies_removed_data["energy_sum"])
Energy_Sum_BP_RD.set_xlim(0,100)
plt.title("Affluent Achievers Sum Energy Boxplot With Anomalies Removed")
plt.xlabel("Total Kilowatts Per Day")

In [ ]:
# getting the average kilowatts per day
# grouping by date and getting the average energy consumption for that day
# this data will be merged with the anomaly data so that the anomaly consumption can be compared with the average consumption
Average_Kilowatts_PerDay = anomalies_removed_data.groupby('date')['energy_sum'].mean()
Average_Kilowatts_PerDay = pd.DataFrame(Average_Kilowatts_PerDay)
Average_Kilowatts_PerDay.rename(columns={'energy_sum':'Average_Energy_That_Day'}, inplace= True)
Average_Kilowatts_PerDay.Average_Energy_That_Day = Average_Kilowatts_PerDay.Average_Energy_That_Day.round(2)
Average_Kilowatts_PerDay.head(5)

In [ ]:
Average_Kilowatts_PerDay.to_csv(r'C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\Average_energy_per_day.csv', index = True)

In [ ]:
# checking if the average energy per day dataframe has 365 rows (1 year)
len(Average_Kilowatts_PerDay)

In [ ]:
# merging anomalies data with average energy per day data
# this was done so that we could compare the actual amount of energy the household used compared to average for each day
# when this data was exported in excel, it helped us determine if the anomaly was an over or under consumption anomaly
anomalies = anomalies.merge(Average_Kilowatts_PerDay, on = 'date').reset_index()
anomalies=anomalies.drop(columns=['level_0'], axis = 1)
anomalies=anomalies.drop(columns=['index'], axis = 1)
anomalies.sort_values(by=["energy_sum"], ascending=False).reset_index()
anomalies.head(5)

In [ ]:
# sorting data in descending order
anomalies =anomalies.sort_values(by=["energy_sum"], ascending=False).reset_index()
anomalies.head(5)

In [ ]:
# checking the number of anomalies
len(anomalies)

In [ ]:
anomalies.to_csv(r'C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\AA_anomalies.csv', index = False)

# Months


In [ ]:
AA_anomalies = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\AA_anomalies.csv")

In [ ]:
len(AA_anomalies)

In [ ]:
# grouping by month and household_id
# this helped us get the number of days per month each household's electricity readings were an anomaly
# then we ordered the data in descending order
grouped = (AA_anomalies.groupby(
   ['month', 'Household_Id']
).agg(
    {
         'date': 'count'
    }
)).sort_values(by="date", ascending = False).reset_index()
grouped= grouped.rename(columns={'date':'days'})
print(grouped.head())

In [ ]:
len(grouped)

In [ ]:
# saving the combined data frame in folder
grouped.to_csv(r'C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\AA_anomalies_months.csv', index = False)

# Rising Prosperity

In [ ]:
block_7 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_7.csv")
block_8 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_8.csv")
block_9 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_9.csv")
block_10 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_10.csv")
block_11 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_11.csv")
block_12 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_12.csv")
block_13 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_13.csv")
block_14 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_14.csv")
block_15 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_15.csv")
block_16 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_16.csv")
block_17 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_17.csv")
block_18 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_18.csv")
block_19 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_19.csv")
block_20 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_20.csv")
block_21 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_21.csv")
block_22 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_22.csv")
block_23 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_23.csv")
block_24 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_24.csv")
block_25 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_25.csv")
block_26 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_26.csv")
block_27 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_27.csv")
block_28 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_28.csv")
block_29 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_29.csv")
block_30 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_30.csv")
block_31 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_31.csv")
block_32 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_32.csv")
block_33 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_33.csv")
block_34 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_34.csv")
block_35 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_35.csv")
block_36 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_36.csv")
block_37 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_37.csv")
block_38 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_38.csv")
block_39 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_39.csv")
block_40 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_40.csv")
block_41 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_41.csv")
block_42 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_42.csv")
block_43 = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\block_43.csv")


In [ ]:
# adding all files in one data frame
Dfs = [block_7,block_8,block_9,block_10,block_11,block_12,block_13,block_14,block_15,block_16,block_17,block_18,block_19,block_20,block_21,block_22,block_23,block_24,block_25,block_26,block_27,block_28,block_29,block_30,block_31,block_32,block_33,block_34,block_35,block_36,block_37,block_38,block_39,block_40,block_41,block_42,block_43]
Combined = pd.concat(Dfs)

In [ ]:
# checking the number of rows in each dataframe
# make sure that the number of rows makes sense
len(Combined)

In [ ]:
# checking that the combined data frame 
Combined.head(5)

In [ ]:
# renaming columns
Combined.rename(columns = {"day" : "date"}, inplace = True)
household_acorn_info.rename(columns = {"day" : "date"}, inplace = True)

In [ ]:
Combined.head(5)

In [ ]:
# merging combined energy readings with household acorn details
Combined =  Combined.merge(household_acorn_info,on='Household_Id').reset_index()

In [ ]:
# checking if only acorns that belong to Affluent Achievers are in the dataframe 
# I saw that households belonging to acorn D were included so I removed them
Combined.Acorn.unique()

In [ ]:
Combined=Combined.loc[Combined['Acorn']!='ACORN-F']
Combined.head(5)

In [ ]:
# now the correct acorn groups of Affluent Achievers remain
Combined.Acorn.unique()

In [ ]:
len(Combined)

In [ ]:
# boxplot with outliers 
Energy_Sum_BP= sns.boxplot(x= Combined["energy_sum"])
Energy_Sum_BP.set_xlim(0,500)
plt.title("Sum Energy Boxplot")
plt.xlabel("Total Kilowatts Per Day")

In [ ]:
# rounding energy_sum to 2 decimal places
Combined.energy_sum = Combined.energy_sum.round(2)
Combined.head(5)

In [ ]:
# skewness and kurtosis values of energy sum column
print ("Skew: %f"%Combined["energy_sum"].skew())
print ("Kurt: %f"%Combined["energy_sum"].kurt())

In [ ]:
# replacing inf values with nan
Combined.replace([np.inf, -np.inf], np.nan)

In [ ]:
# checking how many null values are in the energy_sum column
count_nan = len(Combined) - Combined.count()
print(count_nan)

In [ ]:
# keeping energy_sum rows that are not empty
Combined = Combined[Combined['energy_sum'].notna()]
len(Combined)

In [ ]:
# seeing if null values in energy_sum column were removed
count_nan = len(Combined) - Combined.count()
print(count_nan)

In [ ]:
# running the isolation tree model on the energy_sum column
rp_ad=IsolationForest(n_estimators=60, max_samples='auto', contamination=float(0.1),max_features=1.0)
rp_ad.fit(Combined[['energy_sum']])

In [ ]:
# adding score and anomaly columns to the dataframe
# if the anomaly column has a 1 that means the row is not an anomaly, if it's a -1 that means it's an anomaly
Combined['Score']=rp_ad.decision_function(Combined[['energy_sum']])
Combined['Anomaly']=rp_ad.predict(Combined[['energy_sum']])
Combined.head(5)

In [ ]:
# storing the anomalies in a new dataframe
anomalies=Combined.loc[Combined['Anomaly']==-1].reset_index()
anomalies =anomalies.drop(['index'],axis=1)
anomalies.head(5)

In [ ]:
len(anomalies)

In [ ]:
# sorting the new dataframe in descending order
anomalies.sort_values(by=["energy_sum"], ascending=False).reset_index()

In [ ]:
# percentage of data that are anomalies
print("Percentage of data that are anomalies:",round(len(anomalies)/len(Combined)*100,1),"%")

In [ ]:
anomalies.to_csv(r'C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\RP_anomalies.csv', index = False)

In [ ]:
# storing the data that are not anomalies in a new dataframe
anomalies_removed_data=Combined.loc[Combined['Anomaly']!=-1].reset_index()
anomalies_removed_data =anomalies_removed_data.drop(['index'],axis=1)
anomalies_removed_data.head(5)

In [ ]:
len(anomalies_removed_data)

In [ ]:
# boxplot without anomalies
Energy_Sum_BP_RD= sns.boxplot(x= anomalies_removed_data["energy_sum"])
Energy_Sum_BP_RD.set_xlim(0,100)
plt.title("Sum Energy Boxplot With Anomalies Removed")
plt.xlabel("Total Kilowatts Per Day")

In [ ]:
# getting the average kilowatts per day
# grouping by day and getting the average energy consumption for that day
# this data will be merged with the anomaly data
Average_Kilowatts_PerDay = anomalies_removed_data.groupby('date')['energy_sum'].mean()
Average_Kilowatts_PerDay = pd.DataFrame(Average_Kilowatts_PerDay)
Average_Kilowatts_PerDay.rename(columns={'energy_sum':'Average_Energy_That_Day'}, inplace= True)
Average_Kilowatts_PerDay.Average_Energy_That_Day = Average_Kilowatts_PerDay.Average_Energy_That_Day.round(2)
Average_Kilowatts_PerDay.head(5)

In [ ]:
# merging anomalies data with average energy per day data
# this was done so that we could compare the actual amount of energy the household used compared to average for each day
# when this data was exported in excel, it helped us determine if the anomaly was an over or under consumption anomaly
anomalies = anomalies.merge(Average_Kilowatts_PerDay, on = 'date').reset_index()
anomalies=anomalies.drop(columns=['level_0'], axis = 1)
anomalies=anomalies.drop(columns=['index'], axis = 1)
anomalies.sort_values(by=["energy_sum"], ascending=False).reset_index()
anomalies.head(5)

In [ ]:
# sorting data in descending order
anomalies =anomalies.sort_values(by=["energy_sum"], ascending=False).reset_index()
anomalies.head(5)

In [ ]:
len(anomalies)

In [ ]:
anomalies.to_csv(r'C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\RP_anomalies.csv', index = False)

In [ ]:
RP_anomalies = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\RP_anomalies.csv")

In [ ]:
# removing unnecessary column
RP_anomalies=RP_anomalies.drop(['index'], axis=1)
RP_anomalies.head(5)

In [ ]:
len(RP_anomalies)

In [ ]:
# grouping by month and household 
# this helped us get the number of days per month each household's electricity readings were an anomaly
# then we ordered the data in descending order
grouped = (RP_anomalies.groupby(
   ['month', 'Household_Id']
).agg(
    {
         'date': 'count'
    }
)).sort_values(by="date", ascending = False).reset_index()
grouped= grouped.rename(columns={'date':'days'})
print(grouped.head())

In [ ]:
len(grouped)

In [ ]:
# saving the combined data frame in folder
grouped.to_csv(r'C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\RP_anomalies_months.csv', index = False)

# Additional Community Groups

In [ ]:
AA_Over = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\AA_Over.csv")
AA_Under = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\AA_Under.csv")
RP_Over = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\RP_Over.csv")
RP_Under = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\RP_Under.csv")
CC_Over = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\CC_anomalies_Over.csv")
CC_Under = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\CC_anomalies_Under.csv")
FS_Over = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\FS_anomalies_over_count.csv")
FS_Under = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\FS_anomalies_under_count.csv")
UA_Over = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\UA_anomalies_OVER.csv")
UA_Under = pd.read_csv(r"C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\UA_anomalies_Under.csv")

In [ ]:
len(AA_Over)

In [ ]:
# grouping by month and household 
# this helped us get the number of days per month each household's electricity readings were an anomaly
# then we ordered the data in descending order
AA_g = (AA_Over.groupby(
   ['month', 'Household_Id']
).agg(
    {
         'date': 'count'
    }
)).sort_values(by="date", ascending = False).reset_index()
AA_g= AA_g.rename(columns={'date':'days'})
print(AA_g.head())

In [ ]:
# saving the combined data frame in folder
AA_g.to_csv(r'C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\AA_Over_months.csv', index = False)

In [ ]:
len(AA_Under)

In [ ]:
# grouping by month and household 
# this helped us get the number of days per month each household's electricity readings were an anomaly
# then we ordered the data in descending order
AA_g1 = (AA_Under.groupby(
   ['month', 'Household_Id']
).agg(
    {
         'date': 'count'
    }
)).sort_values(by="date", ascending = False).reset_index()
AA_g1= AA_g1.rename(columns={'date':'days'})
print(AA_g1.head())

In [ ]:
# saving the combined data frame in folder
AA_g1.to_csv(r'C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\AA_Under_months.csv', index = False)

In [ ]:
len(RP_Over)

In [ ]:
# grouping by month and household 
# this helped us get the number of days per month each household's electricity readings were an anomaly
# then we ordered the data in descending order
RP_g = (RP_Over.groupby(
   ['month', 'Household_Id']
).agg(
    {
         'date': 'count'
    }
)).sort_values(by="date", ascending = False).reset_index()
RP_g= RP_g.rename(columns={'date':'days'})
print(RP_g.head())

In [ ]:
# saving the combined data frame in folder
RP_g.to_csv(r'C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\RP_Over_months.csv', index = False)

In [ ]:
len(RP_Under)

In [ ]:
# grouping by month and household 
# this helped us get the number of days per month each household's electricity readings were an anomaly
# then we ordered the data in descending order
RP_g1 = (RP_Under.groupby(
   ['month', 'Household_Id']
).agg(
    {
         'date': 'count'
    }
)).sort_values(by="date", ascending = False).reset_index()
RP_g1= RP_g1.rename(columns={'date':'days'})
print(RP_g1.head())

In [ ]:
# saving the combined data frame in folder
RP_g1.to_csv(r'C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\RP_Under_months.csv', index = False)

In [ ]:
len(CC_Over)

In [ ]:
# grouping by month and household 
# this helped us get the number of days per month each household's electricity readings were an anomaly
# then we ordered the data in descending order
CC_g = (CC_Over.groupby(
   ['month', 'Household_Id']
).agg(
    {
         'date': 'count'
    }
)).sort_values(by="date", ascending = False).reset_index()
CC_g= CC_g.rename(columns={'date':'days'})
print(CC_g.head())

In [ ]:
# saving the combined data frame in folder
CC_g.to_csv(r'C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\CC_Over_months.csv', index = False)

In [ ]:
len(CC_Under)

In [ ]:
# grouping by month and household 
# this helped us get the number of days per month each household's electricity readings were an anomaly
# then we ordered the data in descending order
CC_g1 = (CC_Under.groupby(
   ['month', 'Household_Id']
).agg(
    {
         'date': 'count'
    }
)).sort_values(by="date", ascending = False).reset_index()
CC_g1= CC_g1.rename(columns={'date':'days'})
print(CC_g1.head())

In [ ]:
# saving the combined data frame in folder
CC_g1.to_csv(r'C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\CC_Under_months.csv', index = False)

In [ ]:
len(FS_Over)

In [ ]:
# grouping by month and household 
# this helped us get the number of days per month each household's electricity readings were an anomaly
# then we ordered the data in descending order
FS_g = (FS_Over.groupby(
   ['month', 'Household_Id']
).agg(
    {
         'date': 'count'
    }
)).sort_values(by="date", ascending = False).reset_index()
FS_g= FS_g.rename(columns={'date':'days'})
print(FS_g.head())

In [ ]:
# saving the combined data frame in folder
FS_g.to_csv(r'C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\FS_Over_months.csv', index = False)

In [ ]:
len(FS_Under)

In [ ]:
# grouping by month and household 
# this helped us get the number of days per month each household's electricity readings were an anomaly
# then we ordered the data in descending order
FS_g1 = (FS_Under.groupby(
   ['month', 'Household_Id']
).agg(
    {
         'date': 'count'
    }
)).sort_values(by="date", ascending = False).reset_index()
FS_g1= FS_g1.rename(columns={'date':'days'})
print(FS_g1.head())

In [ ]:
# saving the combined data frame in folder
FS_g1.to_csv(r'C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\FS_Under_months.csv', index = False)

In [ ]:
len(UA_Over)

In [ ]:
# grouping by month and household 
# this helped us get the number of days per month each household's electricity readings were an anomaly
# then we ordered the data in descending order
UA_g = (UA_Over.groupby(
   ['month', 'Household_Id']
).agg(
    {
         'date': 'count'
    }
)).sort_values(by="date", ascending = False).reset_index()
UA_g= UA_g.rename(columns={'date':'days'})
print(UA_g.head())

In [ ]:
# saving the combined data frame in folder
UA_g.to_csv(r'C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\UA_Over_months.csv', index = False)

In [ ]:
len(UA_Under)

In [ ]:
# grouping by month and household 
# this helped us get the number of days per month each household's electricity readings were an anomaly
# then we ordered the data in descending order
UA_g1 = (UA_Under.groupby(
   ['month', 'Household_Id']
).agg(
    {
         'date': 'count'
    }
)).sort_values(by="date", ascending = False).reset_index()
UA_g1= UA_g1.rename(columns={'date':'days'})
print(UA_g1.head())

In [ ]:
# saving the combined data frame in folder
UA_g1.to_csv(r'C:\Users\m\Desktop\Masters Modules\MS5103 Business Analytics Project\Datasets\Smart Meters\Cleaned Data\daily_dataset\UA_Under_months.csv', index = False)